# Imports


In [ ]:
import sys
print(sys.executable)

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

import matplotlib.pyplot as plt

sampling_seed = tfp.random.sanitize_seed(seed=146098)

# Effect of GP kernels


In [ ]:
predictive_index_points_ = np.linspace(-1.2, 1.2, 200, dtype=np.float64)
predictive_index_points_ = predictive_index_points_[..., np.newaxis]

num_samples = 20

gp = tfd.GaussianProcess(
  kernel=tfk.ExponentiatedQuadratic(np.float64(1), np.float64(0.5)),
  index_points=predictive_index_points_,
)

samples = gp.sample(num_samples, seed = sampling_seed)

plt.figure(figsize=(7, 4))
for i in range(num_samples):
    plt.plot(
        predictive_index_points_,
        samples[i, :],
        c="r",
        alpha=0.1 + 0.9 * (i + 1 == num_samples),
        label="Posterior Sample" if i == 0 else None,
    )
leg = plt.legend(loc="lower right")
for lh in leg.legend_handles:
    lh.set_alpha(1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title(r"Gaussian Process Realisation With the Squared Exponential Kernel")
plt.savefig("write_up/images/exponentiated_kernel.pdf")
plt.show()

gp = tfd.GaussianProcess(
  kernel=tfk.MaternOneHalf(np.float64(1), np.float64(0.5)),
  index_points=predictive_index_points_,
)

samples = gp.sample(num_samples, seed = sampling_seed)

plt.figure(figsize=(7, 4))
for i in range(num_samples):
    plt.plot(
        predictive_index_points_,
        samples[i, :],
        c="r",
        alpha=0.1 + 0.9 * (i + 1 == num_samples),
        label="Posterior Sample" if i == 0 else None,
    )
leg = plt.legend(loc="lower right")
for lh in leg.legend_handles:
    lh.set_alpha(1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title(r"Gaussian Process Realisation with Matern Kernel $\nu = 1/2$")
plt.savefig("write_up/images/maternonehalf_kernel.pdf")
plt.show()

gp = tfd.GaussianProcess(
  kernel=tfk.MaternThreeHalves(np.float64(1), np.float64(0.5)),
  index_points=predictive_index_points_,
)

samples = gp.sample(num_samples, seed = sampling_seed)

plt.figure(figsize=(7, 4))
for i in range(num_samples):
    plt.plot(
        predictive_index_points_,
        samples[i, :],
        c="r",
        alpha=0.1 + 0.9 * (i + 1 == num_samples),
        label="Posterior Sample" if i == 0 else None,
    )
leg = plt.legend(loc="lower right")
for lh in leg.legend_handles:
    lh.set_alpha(1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title(r"Gaussian Process Realisation with Matern Kernel $\nu = 3/2$")
plt.savefig("write_up/images/maternthreehalves_kernel.pdf")
plt.show()

gp = tfd.GaussianProcess(
  kernel=tfk.MaternFiveHalves(np.float64(1), np.float64(0.5)),
  index_points=predictive_index_points_,
)

samples = gp.sample(num_samples, seed = sampling_seed)

plt.figure(figsize=(7, 4))
for i in range(num_samples):
    plt.plot(
        predictive_index_points_,
        samples[i, :],
        c="r",
        alpha=0.1 + 0.9 * (i + 1 == num_samples),
        label="Posterior Sample" if i == 0 else None,
    )
leg = plt.legend(loc="lower right")
for lh in leg.legend_handles:
    lh.set_alpha(1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title(r"Gaussian Process Realisation with Matern Kernel $\nu = 5/2$")
plt.savefig("write_up/images/maternfivehalves_kernel.pdf")
plt.show()

## Changing length parameter no regression

In [ ]:
num_samples = 50

for ell in [0.5, 1, 2]:
    gp = tfd.GaussianProcess(
        kernel=tfk.ExponentiatedQuadratic(np.float64(1), np.float64(ell)),
        index_points=predictive_index_points_,
    )

    samples = gp.sample(num_samples, seed=sampling_seed)

    plt.figure(figsize=(7, 4))
    for i in range(num_samples):
        plt.plot(
            predictive_index_points_,
            samples[i, :],
            c="r",
            alpha=0.1 + 0.9 * (i + 1 == num_samples),
            label="Posterior Sample" if i == 0 else None,
        )
    leg = plt.legend(loc="lower right")
    for lh in leg.legend_handles:
        lh.set_alpha(1)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.title(r"Squared Exponential kernel with $\ell = $" + str(ell))
    plt.savefig(
        "write_up/images/exponentiated_GP_ell_" + str(int(ell * 10)) + "_tenths.pdf"
    )
    plt.show()

## Changing length parameter with regression

In [ ]:
flat_index = np.array([[-1], [0], [1]], dtype=np.float64)
easy_obs = np.array([0, 0, 1], dtype=np.float64)

plt.figure(figsize=(7, 4))
plt.scatter(flat_index[:, 0], easy_obs, label="Observations")
leg = plt.legend(loc="lower right")
for lh in leg.legend_handles:
    lh.set_alpha(1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.ylim(-2, 2)
plt.title("Initial Points")
plt.savefig("write_up/images/flatish_GP_bare.pdf")
plt.show()

In [ ]:
predictive_index_points_ = np.linspace(-1.2, 1.2, 200, dtype=np.float64)
predictive_index_points_ = predictive_index_points_[..., np.newaxis]
num_samples = 50

for ell in [0.5, 1, 2]:
    gprm = tfd.GaussianProcessRegressionModel(
        kernel=tfk.ExponentiatedQuadratic(np.float64(1), np.float64(ell)),
        index_points=predictive_index_points_,
        observation_index_points=flat_index,
        observations=easy_obs,
        predictive_noise_variance=0.0,
    )

    samples = gprm.sample(num_samples, seed = sampling_seed)

    plt.figure(figsize=(7, 4))
    for i in range(num_samples):
        plt.plot(
            predictive_index_points_,
            samples[i, :],
            c="r",
            alpha=0.1 + 0.9 * (i + 1 == num_samples),
            label="Posterior Sample" if i == 0 else None,
        )
    plt.scatter(flat_index[:, 0], easy_obs, label="Observations")
    leg = plt.legend(loc="lower right")
    for lh in leg.legend_handles:
        lh.set_alpha(1)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.ylim(-2, 2)
    plt.title(r"Squared Exponential Kernel with $\ell = $" + str(ell))
    plt.savefig("write_up/images/flatish_GP_ell_" + str(int(ell*10)) + "_tenths.pdf")
    plt.show()

## Changing amplitude parameter

In [ ]:
for sigma in [0.5, 1, 2]:
    gprm = tfd.GaussianProcessRegressionModel(
        kernel=tfk.ExponentiatedQuadratic(np.float64(sigma), np.float64(1)),
        index_points=predictive_index_points_,
        observation_index_points=flat_index,
        observations=easy_obs,
        predictive_noise_variance=0.0,
    )

    samples = gprm.sample(num_samples, seed = sampling_seed)

    plt.figure(figsize=(7, 4))
    for i in range(num_samples):
        plt.plot(
            predictive_index_points_,
            samples[i, :],
            c="r",
            alpha=0.1 + 0.9 * (i + 1 == num_samples),
            label="Posterior Sample" if i == 0 else None,
        )
    plt.scatter(flat_index[:, 0], easy_obs, label="Observations")
    leg = plt.legend(loc="lower right")
    for lh in leg.legend_handles:
        lh.set_alpha(1)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.ylim(-2, 2)
    plt.title(r"Function estimation with $\sigma = $" + str(sigma))
    plt.savefig("write_up/images/flatish_GP_sigma_" + str(sigma*10) + "_tenths.pdf")
    plt.show()

## Matern kernels

In [ ]:
predictive_index_points_ = np.linspace(-1.2, 1.2, 400, dtype=np.float64)
predictive_index_points_ = predictive_index_points_[..., np.newaxis]

for nu in [0.5, 1.5, 2.5]:
    gprm = tfd.GaussianProcessRegressionModel(
        kernel=tfk.GeneralizedMatern(nu, np.float64(1), np.float64(1)),
        index_points=predictive_index_points_,
        observation_index_points=flat_index,
        observations=easy_obs,
        predictive_noise_variance=0.0,
    )

    samples = gprm.sample(num_samples / 2, seed=sampling_seed)

    plt.figure(figsize=(7, 4))
    for i in range(int(num_samples / 2)):
        plt.plot(
            predictive_index_points_,
            samples[i, :],
            c="r",
            alpha=0.1 + 0.9 * (i + 1 == int(num_samples / 2)),
            label="Posterior Sample" if i == 0 else None,
        )
    plt.scatter(flat_index[:, 0], easy_obs, label="Observations", alpha=1)
    leg = plt.legend(loc="lower right")
    for lh in leg.legend_handles:
        lh.set_alpha(1)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.ylim(-2, 2)
    plt.title(r"Matern Kernel with $\nu = $" + str(nu))
    plt.savefig("write_up/images/flatish_GP_matern_" + str(nu * 10) + "_tenths.pdf")
    plt.show()

# GP for Noiseless Cubic Target Function


In [ ]:
my_seed = np.random.default_rng(seed=591)  # For replicability


def cub_fn(x):
  return x[..., 0] * (x[..., 0] - 1) * (x[..., 0] + 1)

ind_range = 10
cub_index_vals = np.expand_dims(  # this makes the size of the sample (3,1)
  my_seed.uniform(low=-1.0, high=1.0, size=ind_range), axis=-1  # last axis
)

cub_obs_vals = cub_fn(cub_index_vals)

my_GP = tfd.GaussianProcess(
  kernel=tfk.ExponentiatedQuadratic(
    amplitude=tf.Variable(1.0, dtype=np.float64, name="amplitude"),
    length_scale=tf.Variable(1.0, dtype=np.float64, name="length_scale"),
  ),
  index_points=cub_index_vals,
)

print(my_GP.trainable_variables[1])

Adam_optim = tf.optimizers.Adam()

In [ ]:
@tf.function()
def optimize():
  with tf.GradientTape() as tape:
    loss = -my_GP.log_prob(cub_obs_vals)
  grads = tape.gradient(loss, my_GP.trainable_variables)
  Adam_optim.apply_gradients(zip(grads, my_GP.trainable_variables))
  return loss


num_iters = 7000

lls_ = np.zeros(num_iters, np.float64)
for i in range(num_iters):
  loss = optimize()
  lls_[i] = loss

amp_fin = my_GP.trainable_variables[0].numpy()
len_fin = my_GP.trainable_variables[1].numpy()

print("Trained parameters:")
print("amplitude: {}".format(amp_fin))
print("length_scale: {}".format(len_fin))

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(lls_)
plt.xlabel("Training iteration")
plt.ylabel("Log likelihood")
plt.show()

In [ ]:
predictive_index_points_ = np.linspace(-1.2, 1.2, 200, dtype=np.float64)

# Reshape to [200, 1] -- 1 is the dimensionality of the feature space.
predictive_index_points_ = predictive_index_points_[..., np.newaxis]

optimized_kernel = tfk.ExponentiatedQuadratic(amp_fin, len_fin)
num_samples = 20

for ind in range(ind_range):
  gprm = tfd.GaussianProcessRegressionModel(
    kernel=optimized_kernel,
    index_points=predictive_index_points_,
    observation_index_points=cub_index_vals[range(ind+1)],
    observations=cub_obs_vals[range(ind+1)],
    predictive_noise_variance=0.0,
  )

  samples = gprm.sample(num_samples, seed = sampling_seed)

  plt.figure(figsize=(7, 4))
  plt.plot(predictive_index_points_, cub_fn(predictive_index_points_),
    label='True fn')
  for i in range(num_samples):
    plt.plot(predictive_index_points_, samples[i, :], c='r', alpha=.2,
      label='Posterior Sample' if i == 0 else None)
  plt.scatter(cub_index_vals[range(ind+1)], cub_obs_vals[range(ind+1)],
    label='Observations')
  leg = plt.legend(loc='upper right')
  for lh in leg.legend_handles:
    lh.set_alpha(1)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.ylim(-1, 1)
  # plt.title(r"Function estimation with GP")
  plt.savefig("write_up/images/cub_GP_" + str(ind + 1) + "_iters.pdf")
  plt.show()

# GP for Cubic Target Function with Noise


In [ ]:
num_obs_err = 20

my_seed_err = np.random.default_rng(seed=914)

# cub_obs_vals_err = cub_obs_vals + my_seed.normal(
#         scale=0.1, size=num_obs_err
#     )

def cub_fn_err(x):
    return x[..., 0] * (x[..., 0] - 1) * (x[..., 0] + 1) + my_seed_err.normal(
        scale=0.1, size=x.shape[0]
    )


cub_obs_vals_err = cub_fn_err(cub_index_vals)

for ind in range(ind_range):
    gprm = tfd.GaussianProcessRegressionModel(
        kernel=optimized_kernel,
        index_points=predictive_index_points_,
        observation_index_points=cub_index_vals[range(ind + 1)],
        observations=cub_obs_vals_err[range(ind + 1)],
        observation_noise_variance=tf.Variable(
            0.01, dtype=np.float64, name="observation_noise_variance_err"
        ),
        predictive_noise_variance=0.0,
    )

    samples = gprm.sample(num_samples, seed=sampling_seed)

    plt.figure(figsize=(7, 4))
    plt.plot(
        predictive_index_points_, cub_fn(predictive_index_points_), label="True fn"
    )
    for i in range(num_samples):
        plt.plot(
            predictive_index_points_,
            samples[i, :],
            c="r",
            alpha=0.2,
            label="Posterior Sample" if i == 0 else None,
        )
    plt.scatter(
        cub_index_vals[range(ind + 1)],
        cub_obs_vals_err[range(ind + 1)],
        label="Observations",
    )
    leg = plt.legend(loc="upper right")
    for lh in leg.legend_handles:
        lh.set_alpha(1)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.ylim(-1, 1)
    # plt.title(r"Function estimation with GP")
    plt.savefig("write_up/images/cub_GP_err_" + str(ind + 1) + "_iters.pdf")
    plt.show()

In [ ]:
UCB_index_points = np.array([[0.0]])
UCB_obs_points = cub_fn_err(UCB_index_points)

next_point = tfp.util.TransformedVariable(
    initial_value=0.5,
    bijector=tfb.Sigmoid(np.float64(-1.0), np.float64(1.0)),
    dtype=np.float64,
)

UCB_GP_reg = tfd.GaussianProcessRegressionModel(
    kernel=optimized_kernel,
    observation_index_points=UCB_index_points,
    observations=UCB_obs_points,
    observation_noise_variance=tf.Variable(
        0.01, dtype=np.float64, name="observation_noise_variance_err"
    ),
)


def UCB_loss():
    x = tf.reshape(next_point, [1, 1])
    mean_t = UCB_GP_reg.mean_fn(x)
    std_t = UCB_GP_reg.stddev(index_points=x)
    return tf.squeeze(mean_t - std_t)

def update_var_UCB():
    optimizer_fast = tf.optimizers.Adam(learning_rate=0.01)

    @tf.function(autograph=False, jit_compile=False)
    def opt_var():
        with tf.GradientTape() as tape:
            loss = UCB_loss()
        grads = tape.gradient(loss, next_point.trainable_variables)
        optimizer_fast.apply_gradients(zip(grads, next_point.trainable_variables))
        return loss

    max_iters = 10000

    lls_ = np.zeros(max_iters, np.float64)
    tolerance = 1e-6  # Set your desired tolerance level
    previous_loss = float("inf")

    for i in range(max_iters):
        loss = opt_var()
        lls_[i] = loss

        # Check if change in loss is less than tolerance
        if abs(loss - previous_loss) < tolerance:
            print(f"Acquisition function convergence reached at iteration {i+1}.")
            break

        previous_loss = loss

UCB_loss()

In [ ]:
for t in range(ind_range):


In [ ]:
Adam_optim = tf.optimizers.Adam() # somehow need this again? maybe when you do the optimize function it stores info?

@tf.function()
def optimize_err():
  with tf.GradientTape() as tape:
    loss = -my_GP_err.log_prob(cub_obs_vals_err)
  grads = tape.gradient(loss, my_GP_err.trainable_variables)
  Adam_optim.apply_gradients(zip(grads, my_GP_err.trainable_variables))
  return loss


num_iters_err = 5000

lls_err = np.zeros(num_iters_err, np.float64)
for i in range(num_iters_err):
  loss_err = optimize_err()
  lls_err[i] = loss_err

amp_fin_err = my_GP_err.trainable_variables[0].numpy()
len_fin_err = my_GP_err.trainable_variables[1].numpy()
obs_var_fin_err = my_GP_err.trainable_variables[2].numpy()

print('Trained parameters:')
print('amplitude_err: {}'.format(amp_fin))
print('length_scale_err: {}'.format(len_fin))
print('observation_noise_variance_err: {}'.format(obs_var_fin_err))

plt.figure(figsize=(7, 4))
plt.plot(lls_err)
plt.xlabel("Training iteration")
plt.ylabel("Log likelihood")
plt.show()

In [ ]:
optimized_kernel_err = tfk.ExponentiatedQuadratic(amp_fin_err, len_fin_err)
gprm_err = tfd.GaussianProcessRegressionModel(
  kernel=optimized_kernel_err,
  index_points=predictive_index_points_,
  observation_index_points=cub_index_vals_err,
  observations=cub_obs_vals_err,
  observation_noise_variance=obs_var_fin_err,
  predictive_noise_variance=0.)

num_samples_err = 50
samples_err = gprm_err.sample(num_samples_err, seed = sampling_seed)

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(predictive_index_points_, cub_fn(predictive_index_points_),
  label='True fn')
plt.scatter(cub_index_vals_err[:, 0], cub_obs_vals_err,
  label='Observations')
for i in range(num_samples_err):
  plt.plot(predictive_index_points_, samples_err[i, :], c='r', alpha=.1,
    label='Posterior Sample' if i == 0 else None)
leg = plt.legend(loc='upper right')
for lh in leg.legend_handles:
  lh.set_alpha(1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.show()

# GP with Quadratic Mean Function for Cubic Target Function


In [ ]:
num_obs_quad=4

my_seed_quad=np.random.default_rng(seed=687)


cub_index_vals_quad = np.expand_dims( # this makes the size of the sample (3,1)
  my_seed_quad.uniform(low=-1., high=1., size=num_obs_quad),
  axis=-1 # last axis
)

cub_obs_vals_quad = cub_fn(cub_index_vals_quad)

def custom_mean_fn(x):
  # You can define your own mean function here
  # For example, a linear mean function: return 2.0 * x
  return - x[..., 0]**2

my_GP_quad = tfd.GaussianProcess(
  kernel=tfk.ExponentiatedQuadratic(
    amplitude=tf.Variable(1., dtype=np.float64, name="amplitude_quad"),
    length_scale=tf.Variable(1., dtype=np.float64, name="length_scale_quad")
  ),
  index_points=cub_index_vals_quad,
  mean_fn=custom_mean_fn)

print(my_GP_quad.trainable_variables)

In [ ]:
Adam_optim = tf.optimizers.Adam() # somehow need this again? maybe when you do the optimize function it stores info?

@tf.function()
def optimize_quad():
  with tf.GradientTape() as tape:
    loss = -my_GP_quad.log_prob(cub_obs_vals_quad)
  grads = tape.gradient(loss, my_GP_quad.trainable_variables)
  Adam_optim.apply_gradients(zip(grads, my_GP_quad.trainable_variables))
  return loss


num_iters_quad = 5000

lls_quad = np.zeros(num_iters_quad, np.float64)
for i in range(num_iters_quad):
  loss_quad = optimize_quad()
  lls_quad[i] = loss_quad

amp_fin_quad = my_GP_quad.trainable_variables[0].numpy()
len_fin_quad = my_GP_quad.trainable_variables[1].numpy()

print('Trained parameters:')
print('amplitude_quad: {}'.format(amp_fin))
print('length_scale_quad: {}'.format(len_fin))

plt.figure(figsize=(7, 4))
plt.plot(lls_quad)
plt.xlabel("Training iteration")
plt.ylabel("Log likelihood")
plt.show()

In [ ]:
optimized_kernel_quad = tfk.ExponentiatedQuadratic(amp_fin_quad, len_fin_quad)
gprm_quad = tfd.GaussianProcessRegressionModel(
  kernel=optimized_kernel_quad,
  index_points=predictive_index_points_,
  observation_index_points=cub_index_vals_quad,
  observations=cub_obs_vals_quad,
  predictive_noise_variance=0.,
  mean_fn=custom_mean_fn)

num_samples_quad = 50
samples_quad = gprm_quad.sample(num_samples_quad, seed = sampling_seed)

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(predictive_index_points_, cub_fn(predictive_index_points_),
  label='True fn')
plt.scatter(cub_index_vals_quad[:, 0], cub_obs_vals_quad,
  label='Observations')
for i in range(num_samples_quad):
  plt.plot(predictive_index_points_, samples_quad[i, :], c='r', alpha=.1,
    label='Posterior Sample' if i == 0 else None)
leg = plt.legend(loc='upper right')
for lh in leg.legend_handles:
  lh.set_alpha(1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.show()

# GP for 2 observations with Differing Observation Variance


In [ ]:
def varying_variance(num_low, num_high, var_low, var_high):
  seed_low=np.random.default_rng(seed=973)
  seed_high=np.random.default_rng(seed=8973)
  seed_other=np.random.default_rng(seed=873)

  samples_low_ = seed_low.normal(scale = np.sqrt(var_low), size = num_low) #+ np.ones(num_low)
  samples_high_ = seed_high.normal(scale = np.sqrt(var_high), size = num_high) + np.ones(num_high)

  x_indices_ = np.expand_dims( # this makes the size of the sample (3,1)
    np.concatenate((np.zeros(num_low), np.ones(num_high))),
    axis=-1 # last axis
    )
  y_vals_ = np.concatenate((samples_low_, samples_high_))

  GP_ = tfd.GaussianProcess(
    kernel=tfk.ExponentiatedQuadratic(
      amplitude=tf.Variable(1., dtype=np.float64, name="amplitude"),
      length_scale=tf.Variable(1., dtype=np.float64, name="length_scale")
    ),
    observation_noise_variance=tf.Variable(1., dtype=np.float64, name="observation_noise_variance"),
    index_points=x_indices_
  )

  optimizer_ = tf.optimizers.Adam() # somehow need this again? maybe when you do the optimize function it stores info?

  @tf.function()
  def optimize_():
    with tf.GradientTape() as tape:
      loss = -GP_.log_prob(y_vals_)
    grads = tape.gradient(loss, GP_.trainable_variables)
    optimizer_.apply_gradients(zip(grads, GP_.trainable_variables))
    return loss


  num_iters_ = 5000

  lls_ = np.zeros(num_iters_, np.float64)
  for i in range(num_iters_):
    loss_ = optimize_()
    lls_[i] = loss_

  amp_fin_ = GP_.trainable_variables[0].numpy()
  len_fin_ = GP_.trainable_variables[1].numpy()
  noise_fin_ = GP_.trainable_variables[2].numpy()

  predictive_index_points_ = np.linspace(-.2, 1.2, 20, dtype=np.float64)
  # Reshape to [200, 1] -- 1 is the dimensionality of the feature space.
  predictive_index_points_ = predictive_index_points_[..., np.newaxis]

  optimized_kernel_ = tfk.ExponentiatedQuadratic(amp_fin_, len_fin_)
  gprm_ = tfd.GaussianProcessRegressionModel(
    kernel=optimized_kernel_,
    index_points=predictive_index_points_,
    observation_index_points=x_indices_,
    observations=y_vals_,
    observation_noise_variance=noise_fin_,
    predictive_noise_variance=0.)

  num_fn_samples_ = 50
  tf.random.set_seed(25626)
  samples_ = gprm_.sample(num_fn_samples_, seed = sampling_seed)
  plt.figure(figsize=(7, 4))
  plt.scatter(x_indices_[:, 0], y_vals_,
    label='Observations')
  for i in range(num_fn_samples_):
    plt.plot(predictive_index_points_, samples_[i, :], c='r', alpha=.1,
      label='Posterior Sample' if i == 0 else None)
  leg = plt.legend(loc='upper left')
  for lh in leg.legend_handles:
    lh.set_alpha(1)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.show()

  return amp_fin_, len_fin_, noise_fin_

varying_variance(100, 2, 0.01, 1)

## Experimenting with Differing Numbers of Observations


In [ ]:
varying_variance(3, 3, 0.01, 1)
varying_variance(3, 10, 0.01, 1)
varying_variance(3, 100, 0.01, 1)
varying_variance(10, 3, 0.01, 1)
varying_variance(100, 3, 0.01, 1)